##### Урок 4. Парсинг HTML. XPath
Выберите веб-сайт с табличными данными, который вас интересует.
Напишите код Python, использующий библиотеку requests для отправки HTTP GET-запроса на сайт и получения HTML-содержимого страницы.
Выполните парсинг содержимого HTML с помощью библиотеки lxml, чтобы извлечь данные из таблицы.
Сохраните извлеченные данные в CSV-файл с помощью модуля csv.

Для работы с табличными данными был выбран сайт https://health-diet.ru Этот ресурс содержит много информации в табличном виде. В частности страница с таблицей КБЖУ фастфуда.

In [79]:
import requests
from lxml import html
import pandas as pd

Извлечение данных

In [80]:
# Создаем запрос на страницу
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

url = "https://health-diet.ru"

response = requests.get(url + "/base_of_food/food_24529", headers=headers)

# При получении ответа преобразуем содержимое страницы
if response.status_code == 200:
    print(f"Server response {response.status_code}")
    tree = html.fromstring(response.content)
 
    
else:
    print(f"Error {response.status_code}")

Server response 200


Парсинг данных из таблицы

In [81]:
# Извлечение данных таблицы

all_products = tree.xpath("//div//table/tbody/tr")

fast_food = []
for food in all_products:
    fast_food_nutrients = {}
    # Извлечение данных из строки таблицы
    product = food.xpath(".//a/text()")
    calories = food.xpath("./td[@class='uk-text-right'][1]/text()")
    proteins = food.xpath("./td[@class='uk-text-right'][2]/text()")
    fats = food.xpath("./td[@class='uk-text-right'][3]/text()")
    carbs = food.xpath("./td[@class='uk-text-right'][4]/text()")

# Форимируем словать с данными по покаждому продукту и добавляем в общий список
    
    fast_food_nutrients["product"] = product[0]
    fast_food_nutrients["calories"] = calories[0]
    fast_food_nutrients["proteins"] = proteins[0]
    fast_food_nutrients["fats"] = fats[0]
    fast_food_nutrients["carbs"] = carbs[0]

    fast_food.append(fast_food_nutrients)


Создание датафрейма и обработка данных

In [82]:
# Создание датафрейма
df = pd.DataFrame(fast_food)
df.head()

,product,calories,proteins,fats,carbs
0,"BURGER KING, ванильный коктейль",168 кКал,"3,19 г","8,74 г","19,03 г"
1,"BURGER KING, гамбургер",261 кКал,"14,85 г","10,55 г","25,76 г"
2,"BURGER KING, двойной ""WHOPPER"", без сыра",252 кКал,"13,94 г","15,66 г","12,34 г"
3,"BURGER KING, двойной ""WHOPPER"", с сыром",266 кКал,"14,47 г","17,06 г","11,92 г"
4,"BURGER KING, драники",302 кКал,"2,8 г","19,3 г","27,07 г"


In [83]:
# Проверка на пустые значения 
display(df.isnull().sum())

# Проверка на типы данных
display(df.info())

product     0
calories    0
proteins    0
fats        0
carbs       0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   product   227 non-null    object
 1   calories  227 non-null    object
 2   proteins  227 non-null    object
 3   fats      227 non-null    object
 4   carbs     227 non-null    object
dtypes: object(5)
memory usage: 9.0+ KB


None

In [84]:
# Преобразование категории 'calories' к типу данных int
df['calories'] = df['calories'].apply(lambda x: x.split()[0])
df['calories'] = df['calories'].astype(int)

# Преобразование категории 'proteins' к типу данных float
df['proteins'] = df['proteins'].apply(lambda x: x.split()[0].replace(',','.'))
df['proteins'] = df['proteins'].astype(float)

# Преобразование категории 'fats' к типу данных float
df['fats'] = df['fats'].apply(lambda x: x.split()[0].replace(',','.'))
df['fats'] = df['fats'].astype(float)

# Преобразование категории 'carbs' к типу данных float
df['carbs'] = df['carbs'].apply(lambda x: x.split()[0].replace(',','.'))
df['carbs'] = df['carbs'].astype(float)



In [85]:
# Проверка корректности обработки
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   product   227 non-null    object 
 1   calories  227 non-null    int32  
 2   proteins  227 non-null    float64
 3   fats      227 non-null    float64
 4   carbs     227 non-null    float64
dtypes: float64(3), int32(1), object(1)
memory usage: 8.1+ KB


In [ ]:
# Сохранение информации в файл csv
df.to_csv('fast_food.csv')